## 네이버 블로그에서 본문 수집

### xlsx 파일 읽기

In [ ]:
# 관련 라이브러리를 호출합니다.
import os
import pandas as pd

In [ ]:
# 현재 작업경로를 확인합니다.
os.getcwd()

In [ ]:
# data 폴더로 작업경로를 변경합니다.
os.chdir(path = '../data')

In [ ]:
# 현재 작업경로에 저장된 폴더명과 파일명을 출력합니다.
os.listdir()

In [ ]:
# xlsx 파일을 읽고 데이터프레임을 생성합니다.
cafe = pd.read_excel(io = 'Naver Cafe Links.xlsx')

In [ ]:
# xlsx 파일의 처음 5행을 출력합니다.
cafe.head()

### HTTP 요청 실행

- 네이버 카페글 링크는 아래 형태로 되어 있습니다.<br>
    https://apis.naver.com/cafe-web/cafe-articleapi/cafes/{clubid}/articles/{articleid}

In [ ]:
# 첫 번째 행의 clubid와 articleid를 각각 생성합니다.
i = 1
clubid = cafe['clubid'][i]
articleid = cafe['articleid'][i]

In [ ]:
# 요청 URL을 조립합니다.
URL = f'https://apis.naver.com/cafe-web/cafe-articleapi/cafes/{clubid}/articles/{articleid}'
print(URL)

In [ ]:
# 관련 라이브러리를 호출합니다.
import requests

In [ ]:
# HTTP 요청을 실행합니다. 
res = requests.get(
    url = URL,
    headers = dict(referer = 'https://section.cafe.naver.com/ca-fe/home/search/articles')
)

### HTTP 응답 확인

In [ ]:
# HTTP 응답 상태코드를 확인합니다.
res.status_code

In [ ]:
# HTTP 응답 헤더를 확인합니다. [참고] Content-Type이 'json'입니다.
res.headers

In [ ]:
# HTTP 요청 URL을 확인합니다.
res.url

In [ ]:
# HTTP 응답 바디(str)의 일부를 출력합니다.
res.text[:1000]

### JSON 데이터 처리

In [ ]:
# 관련 라이브러리를 호출합니다.
import json

In [ ]:
# HTTP 응답 바디를 딕셔너리로 변환합니다. 
data = json.loads(s = res.text)

In [ ]:
# data에서 카페글 본문을 body에 할당합니다.
body = data['article']['content']
body

In [ ]:
# 관련 라이브러리를 호출합니다.
import re

In [ ]:
# 정규표현식으로 태그 등을 삭제합니다.
pat = re.compile(r'<.+?>|\[.+?\]|\]|&#8203;')
text = re.sub(pattern = pat, repl = '', string = body)
text

### for 반복문 실행

In [ ]:
# 관련 라이브러리를 호출합니다.
import tqdm, time

In [ ]:
# cafe의 행 길이를 설정합니다.(반복문 실행 횟수)
n = cafe.shape[0]
print(n)

In [ ]:
# 카페글 본문을 저장할 contents 열을 생성하고 빈 문자열을 할당합니다.
cafe['contents'] = ''

In [ ]:
# for 반복문을 실행합니다.
for i in tqdm.tqdm_notebook(range(n)):
    
    # 각 행의 clubid와 articleid를 각각 생성합니다.
    clubid = cafe['clubid'][i]
    articleid = cafe['articleid'][i]
    
    # URL을 조립합니다.
    URL = f'https://apis.naver.com/cafe-web/cafe-articleapi/cafes/{clubid}/articles/{articleid}'
    
    # HTTP 요청을 실행합니다. 
    res = requests.get(
        url = URL,
        headers = dict(referer = 'https://section.cafe.naver.com/ca-fe/home/search/articles')
    )
    
    # HTTP 응답 바디를 딕셔너리로 변환합니다. 
    data = json.loads(s = res.text)
    
    # 카페글 본문이 없으면 에러가 발생하므로 예외처리합니다.
    try:
        
        # data에서 카페글 본문을 body에 할당합니다.
        body = data['article']['content']
        
        # 정규표현식으로 태그 등을 삭제합니다.
        pat = re.compile(r'<.+?>|\[.+?\]|\]|&#8203;')
        text = re.sub(pattern = pat, repl = '', string = body)
        
    except:
        
        print(f'{i}번 카페글 본문이 없습니다!')
        text = ''
    
    # contents 열에 text를 추가합니다.
    cafe.loc[i, 'contents'] = text
    
    # 1초간 멈춥니다.
    time.sleep(1)
    

In [ ]:
# cafe의 처음 5행을 출력합니다.
cafe.head()

In [ ]:
# cafe의 정보를 확인합니다.
cafe.info()

In [ ]:
# articleid와 clubid를 문자열로 변환합니다.
cafe = cafe.astype({'articleid': 'str', 'clubid': 'str'})

### xlsx 파일로 저장

In [ ]:
# xlsx 파일명을 설정합니다.
fileName = 'Naver Cafe Contents.xlsx'

In [ ]:
# xlsx 파일로 저장합니다.
cafe.to_excel(excel_writer = fileName, index = None)

# End of Document